In [1]:
import pandas as pd
import numpy as np

In [2]:
import zipfile

In [3]:
url = 'data/kaggle-survey-2018.zip'

In [5]:
with zipfile.ZipFile(url) as z:
    print(z.namelist())
    kag = pd.read_csv(z.open('multipleChoiceResponses.csv'))
    df = kag.iloc[1:]

['multipleChoiceResponses.csv', 'freeFormResponses.csv', 'SurveySchema.csv']


/var/folders/j_/hylrqd052y9czkfmjyyc7n7r0000gn/T/ipykernel_7419/3418348685.py:3: DtypeWarning: Columns (0,2,8,10,21,23,24,25,26,27,28,44,56,64,83,85,87,107,109,123,125,150,157,172,174,194,210,218,219,223,246,249,262,264,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,304,306,325,326,329,341,368,371,384,385,389,390,391,393,394) have mixed types. Specify dtype option on import or set low_memory=False.
  kag = pd.read_csv(z.open('multipleChoiceResponses.csv'))


In [6]:
df.T

,1,2,3,4,5,6,7,8,9,10,...,23850,23851,23852,23853,23854,23855,23856,23857,23858,23859
Time from Start to Finish (seconds),710,434,718,621,731,1142,959,1758,641,751,...,820,683,57,122,348,575,131,370,36,502
Q1,Female,Male,Female,Male,Male,Male,Male,Male,Male,Male,...,Female,Male,Female,Female,Male,Male,Female,Male,Male,Male
Q1_OTHER_TEXT,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
Q2,45-49,30-34,30-34,35-39,22-24,25-29,35-39,18-21,25-29,30-34,...,18-21,22-24,18-21,30-34,30-34,45-49,25-29,22-24,25-29,25-29
Q3,United States of America,Indonesia,United States of America,United States of America,India,Colombia,Chile,India,Turkey,Hungary,...,India,Turkey,Turkey,Turkey,Turkey,France,Turkey,Turkey,United Kingdom of Great Britain and Northern I...,Spain
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q50_Part_5,NaN,NaN,NaN,Not enough incentives to share my work,Not enough incentives to share my work,NaN,NaN,Not enough incentives to share my work,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q50_Part_6,NaN,NaN,NaN,NaN,NaN,I had never considered making my work easier f...,I had never considered making my work easier f...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q50_Part_7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q50_Part_8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.dtypes

Time from Start to Finish (seconds)    object
Q1                                     object
Q1_OTHER_TEXT                          object
Q2                                     object
Q3                                     object
                                        ...  
Q50_Part_5                             object
Q50_Part_6                             object
Q50_Part_7                             object
Q50_Part_8                             object
Q50_OTHER_TEXT                         object
Length: 395, dtype: object

In [8]:
df.Q1.value_counts(dropna=False)

Male                       19430
Female                      4010
Prefer not to say            340
Prefer to self-describe       79
Name: Q1, dtype: int64

In [10]:
df.Q2.value_counts(dropna=False)

25-29    6159
22-24    5141
30-34    3776
18-21    3037
35-39    2253
40-44    1360
45-49     858
50-54     582
55-59     328
60-69     273
70-79      53
80+        39
Name: Q2, dtype: int64

In [14]:
def tweak_kag(df):
    na_mask = df.Q9.isna()
    hide_mask = df.Q9.str.startswith('I do not').fillna(False)
    df = df[~na_mask & ~hide_mask]

    q1 = (
        df.Q1
        .replace({'Prefer not to say': 'Another',
                  'Prefer to self-describe': 'Another'})
        .rename('Gender')
    )

    q2 = df.Q2.str.slice(0, 2).astype(int).rename('Age')

    def limit_countries(val):
        if val in {'United States of America', 'India', 'China'}:
            return val
        return 'Another'
    q3 = df.Q3.apply(limit_countries).rename('Country')

    q4 = (
        df.Q4
        .replace({
            "Master’s degree": 18,
            "Bachelor’s degree": 16,
            "Doctoral degree": 20,
            "Some college/university study without earning a bachelor’s degree": 13,
            "Professional degree": 19,
            "I prefer not to answer": None,
            "No formal education past high school": 12
        })
        .fillna(11)
        .rename('Edu')
    )

    def only_cs_stat_val(val):
        if val not in {'cs', 'eng', 'stat'}:
            return 'another'
        return val

    q5 = (
        df.Q5
        .replace({
            'Computer science (software engineering, etc.)': 'cs',
            'Engineering (non-computer focused)': 'eng',
            'Mathematics or statistics': 'stat'
        })
        .apply(only_cs_stat_val)
        .rename('Studies')
    )

    def limit_occupation(val):
        if val in {'Student', 'Data Scientist', 'Software Engineer', 'Not employed', 'Data Engineer'}:
            return val
        return 'Another'

    q6 = df.Q6.apply(limit_occupation).rename('Occupation')

    q8 = (
        df.Q8
        .str.replace('+', '')
        .str.split('-', expand=True)
        .iloc[:, 0]
        .fillna(-1)
        .astype(int)
        .rename('Experience')
    )

    q9 = (
        df.Q9
        .str.replace('+', '')
        .str.replace(',', '')
        .str.replace('500000', '500')
        .str.replace('I do not wish to disclose my approximate yearly compensation', '')
        .str.split('-', expand=True)
        .iloc[:, 0]
        .astype(int)
        .mul(1000)
        .rename('Salary')
    )

    return pd.concat([q1, q2, q3, q4, q5, q6, q8, q9], axis=1)

In [15]:
tweak_kag(df)

/var/folders/j_/hylrqd052y9czkfmjyyc7n7r0000gn/T/ipykernel_7419/2451101935.py:61: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace('+', '')
/var/folders/j_/hylrqd052y9czkfmjyyc7n7r0000gn/T/ipykernel_7419/2451101935.py:71: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace('+', '')


,Gender,Age,Country,Edu,Studies,Occupation,Experience,Salary
2,Male,30,Another,16.0,eng,Another,5,10000
3,Female,30,United States of America,18.0,cs,Data Scientist,0,0
5,Male,22,India,18.0,stat,Another,0,0
7,Male,35,Another,20.0,another,Another,10,10000
8,Male,18,India,18.0,another,Another,0,0
...,...,...,...,...,...,...,...,...
23844,Male,30,Another,18.0,cs,Software Engineer,10,90000
23845,Male,22,Another,18.0,stat,Student,0,0
23854,Male,30,Another,20.0,cs,Another,5,10000
23855,Male,45,Another,20.0,cs,Another,5,250000


In [16]:
kag = tweak_kag(df)

/var/folders/j_/hylrqd052y9czkfmjyyc7n7r0000gn/T/ipykernel_7419/2451101935.py:61: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace('+', '')
/var/folders/j_/hylrqd052y9czkfmjyyc7n7r0000gn/T/ipykernel_7419/2451101935.py:71: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace('+', '')


In [17]:
(
    kag
    .groupby('Country')
    .apply(lambda g: g.Salary.corr(g.Experience))
)

Country
Another                     0.289827
China                       0.252974
India                       0.167335
United States of America    0.354125
dtype: float64

In [19]:
def limit_countries(val):
    if val in {'United States of America', 'India', 'China'}:
        return val
    return 'Another'

In [29]:
q3 = df.Q3.apply(limit_countries).rename('Country')

In [21]:
%%timeit
values = {'United States of America', 'India', 'China'}
q3_3 = df.Q3.where(df.Q3.isin(values), 'Another')

831 µs ± 7.28 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [22]:
%%timeit
values = {'United States of America', 'India', 'China'}
q3_4 = pd.Series(np.where(df.Q3.isin(values), df.Q3, 'Another'), index=df.index)

753 µs ± 4.99 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [26]:
def debug(something):
    print(type(something), something)

In [30]:
q3.apply(debug)

<class 'str'> United States of America
<class 'str'> Another
<class 'str'> United States of America
<class 'str'> United States of America
<class 'str'> India
<class 'str'> Another
<class 'str'> Another
<class 'str'> India
<class 'str'> Another
<class 'str'> Another
<class 'str'> Another
<class 'str'> United States of America
<class 'str'> Another
<class 'str'> United States of America
<class 'str'> Another
<class 'str'> Another
<class 'str'> Another
<class 'str'> Another
<class 'str'> United States of America
<class 'str'> Another
<class 'str'> Another
<class 'str'> Another
<class 'str'> United States of America
<class 'str'> Another
<class 'str'> Another
<class 'str'> Another
<class 'str'> India
<class 'str'> United States of America
<class 'str'> Another
<class 'str'> Another
<class 'str'> India
<class 'str'> Another
<class 'str'> Another
<class 'str'> China
<class 'str'> Another
<class 'str'> Another
<class 'str'> India
<class 'str'> Another
<class 'str'> Another
<class 'str'> Unit

1        None
2        None
3        None
4        None
5        None
         ... 
23855    None
23856    None
23857    None
23858    None
23859    None
Name: Country, Length: 23859, dtype: object

In [31]:
the_item = None

In [32]:
def debug(something):
    global the_item
    the_item = something
    return something

In [33]:
_ = q3.apply(debug)

In [34]:
the_item

'Another'

In [35]:
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [36]:
%%timeit
res_p = df.Q3.parallel_apply(limit_countries).rename('Country')

76 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
import swifter

In [38]:
%%timeit
res_s = df.Q3.swifter.apply(limit_countries).rename('Country')

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/23859 [00:00<?, ?it/s]

20.8 ms ± 906 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
import dask

In [40]:
%%timeit
res_d = (
    dask.dataframe.from_pandas(
        df, npartitions=4
    )
    .map_partitions(lambda df: df.Q3.apply(limit_countries))
    .rename('Countries')
)

267 ms ± 4.71 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
np_fn = np.vectorize(limit_countries)

In [42]:
%%timeit
res_v = df.Q3.apply(np_fn).rename('Country')

77.2 ms ± 1.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [43]:
from numba import jit

In [44]:
@jit
def limit_countries2(val):
    if val in {'United States of America', 'India', 'China'}:
        return val
    return 'Another'

In [49]:
%%timeit
res_n = df.Q3.apply(limit_countries2).rename('Country')

32.3 ms ± 363 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


코드 검사

In [57]:
df.Q3.apply()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [68]:
from IPython.core.debugger import set_trace

In [69]:
def add1(x):
    set_trace()
    return x + 1

In [70]:
df.Q3.apply(add1)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/Users/taemin/anaconda3/lib/python3.10/bdb.py", line 336, in set_trace
    sys.settrace(self.trace_dispatch)



> /var/folders/j_/hylrqd052y9czkfmjyyc7n7r0000gn/T/ipykernel_7419/2147203382.py(3)add1()
      1 def add1(x):
      2     set_trace()
----> 3     return x + 1

      Time from Start to Finish (seconds)      Q1 Q1_OTHER_TEXT     Q2  \
1                                     710  Female            -1  45-49   
2                                     434    Male            -1  30-34   
3                                     718  Female            -1  30-34   
4                                     621    Male            -1  35-39   
5                                     731    Male            -1  22-24   
...                                   ...     ...           ...    ...   
23855                                 575    Male            -1  45-49   
23856                                 131  Female            -1  25-29   
23857                                 370    Male            -1  22-24   
23858                                  36    Male            -1  25-29   
23859                     


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/Users/taemin/anaconda3/lib/python3.10/bdb.py", line 361, in set_quit
    sys.settrace(None)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/Users/taemin/anaconda3/lib/python3.10/site-packages/IPython/core/debugger.py", line 973, in cmdloop
    sys.settrace(None)



--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user


TypeError: can only concatenate str (not "int") to str

In [65]:
%debug

TypeError: 'PyDB' object is not callable

Great Expectations 를 이용한 데이터 무결성 관리

In [71]:
kag = tweak_kag(df)

In [72]:
import great_expectations as ge

In [73]:
kag_ge = ge.from_pandas(kag)

In [74]:
sorted([x for x in set(dir(kag_ge)) - set(dir(kag)) if not x.startswith('_')])

['add_citation',
 'attempt_allowing_relative_error',
 'autoinspect',
 'batch_id',
 'batch_kwargs',
 'batch_markers',
 'batch_parameters',
 'caching',
 'column_aggregate_expectation',
 'column_map_expectation',
 'column_pair_map_expectation',
 'default_expectation_args',
 'discard_failing_expectations',
 'discard_subset_failing_expectations',
 'edit_expectation_suite',
 'expect_column_bootstrapped_ks_test_p_value_to_be_greater_than',
 'expect_column_chisquare_test_p_value_to_be_greater_than',
 'expect_column_distinct_values_to_be_in_set',
 'expect_column_distinct_values_to_contain_set',
 'expect_column_distinct_values_to_equal_set',
 'expect_column_kl_divergence_to_be_less_than',
 'expect_column_max_to_be_between',
 'expect_column_mean_to_be_between',
 'expect_column_median_to_be_between',
 'expect_column_min_to_be_between',
 'expect_column_most_common_value_to_be_in_set',
 'expect_column_pair_cramers_phi_value_to_be_less_than',
 'expect_column_pair_values_A_to_be_greater_than_B',
 'exp

In [75]:
kag_ge.expect_column_to_exist('Salary')

{
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {},
  "success": true
}

In [76]:
kag_ge.expect_column_mean_to_be_between(
    'Salary', min_value=10_000, max_value=100_000
)

{
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": 43869.66102793441,
    "element_count": 15429,
    "missing_count": null,
    "missing_percent": null
  },
  "success": true
}

In [77]:
kag_ge.expect_column_values_to_not_be_null('Salary')

{
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 15429,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

In [78]:
kag_ge.expect_column_values_to_match_regex(
    'Country', r'America|India|Another|China'
)

{
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "element_count": 15429,
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0,
    "partial_unexpected_list": []
  },
  "success": true
}

In [79]:
kag_ge.expect_column_values_to_be_of_type(
    'Salary', type_='int'
)

{
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  },
  "result": {
    "observed_value": "int64"
  },
  "success": true
}

In [80]:
kag_ge.save_expectation_suite('kaggle_expectations.json')

In [81]:
kag_ge.to_csv('kag.csv')

In [82]:
import json

In [83]:
ge.validate(ge.read_csv('kag.csv'), expectation_suite=json.load(open('kaggle_expectations.json')))

{
  "meta": {
    "great_expectations_version": "0.17.0",
    "expectation_suite_name": "default",
    "run_id": {
      "run_time": "2023-06-22T11:04:50.594729+09:00",
      "run_name": null
    },
    "batch_kwargs": {
      "ge_batch_id": "2bd09658-10a1-11ee-9f71-86325a1eeea7"
    },
    "batch_markers": {},
    "batch_parameters": {},
    "validation_time": "20230622T020450.594573Z",
    "expectation_suite_meta": {
      "great_expectations_version": "0.17.0"
    }
  },
  "statistics": {
    "evaluated_expectations": 5,
    "successful_expectations": 5,
    "unsuccessful_expectations": 0,
    "success_percent": 100.0
  },
  "results": [
    {
      "meta": {},
      "exception_info": {
        "raised_exception": false,
        "exception_message": null,
        "exception_traceback": null
      },
      "result": {},
      "expectation_config": {
        "kwargs": {
          "column": "Salary"
        },
        "meta": {},
        "expectation_type": "expect_column_to_exist"
   